<a href="https://colab.research.google.com/github/Sneha-mav/Churn-Prediction-AutoML/blob/main/telco_churn_H2OAutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h5>Command to install H2O-3 module for Python.</h5>

In [1]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 4.8 MB/s eta 0:00:00


<h5>Importing h2o and H2OAutoML class<h5>

In [4]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

For this experimental H2OAutoml classification demo, I am using [Telco churn dataset](https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv) which is also known as IBM churn dataset.
<br>
The Goal here is to get comparitive analysis between State of arts results perfomed by several papers verus AutoML
<br>
[Paper Links 1](https://www.researchgate.net/publication/381544028_Telco_Customer_Churn_Prediction)<br>
[Link to several papers](https://www.sciencedirect.com/science/article/pii/S2666720723001443#sec0021)

In [3]:
df_churn = pd.read_csv("https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv")

In [5]:
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
df_churn["TotalCharges"] = pd.to_numeric(df_churn["TotalCharges"], errors="coerce").fillna(0)

<h5>Intialising h2o cluster</h5>

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppt2pm74h
  JVM stdout: /tmp/tmppt2pm74h/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmppt2pm74h/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_unknownUser_rw7g6e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [8]:
df_churn_h2o=h2o.H2OFrame(df_churn)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
df_churn_h2o.describe()

Rows:7043
Cols:21

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
type,string,enum,int,enum,enum,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,enum
mins,NaN,,0.0,,,0.0,,,,,,,,,,,,,18.25,0.0,
mean,NaN,,0.1621468124378816,,,32.37114865824243,,,,,,,,,,,,,64.76169246059902,2279.7343035638214,
maxs,NaN,,1.0,,,72.0,,,,,,,,,,,,,118.75,8684.8,
sigma,NaN,,0.3686116056100131,,,24.559481023094442,,,,,,,,,,,,,30.09004709767848,2266.7944696890245,
zeros,0,,5901,,,11,,,,,,,,,,,,,0,11,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,7590-VHVEG,Female,0.0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0.0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0.0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


Identify response column and labeling it as y and rest columns will be used a inputs.<br>
Spliting the data into two frames, a train (80%) and a test frame (20%). The test frame will be used to score the leaderboard and to demonstrate how to generate predictions using an AutoML leader model.

In [10]:
y="Churn"
split=df_churn_h2o.split_frame(ratios=[0.8],seed=42)
train=split[0]
test=split[1]

You can cap the AutoML run to x seconds using max_runtime_secs. Just keep in mind that the number of models trained in that time may differ from run to run, especially if your system’s hardware or workload conditions change.<br>
Here at first its set to be 30s

In [11]:
churn_aml=H2OAutoML(seed=42,max_runtime_secs=30,project_name="telco_churn_aml")
churn_aml_train=churn_aml.train(y=y,training_frame=train,leaderboard_frame=test)

AutoML progress: |
11:03:05.756: _train param, Dropping bad and constant columns: [customerID]

██████████████████
11:03:16.37: _train param, Dropping bad and constant columns: [customerID]

██████████████
11:03:21.780: _train param, Dropping bad and constant columns: [customerID]

████████████████████
11:03:30.966: _train param, Dropping unused columns: [customerID]

████
11:03:32.751: _train param, Dropping bad and constant columns: [customerID]

█████
11:03:34.770: _train param, Dropping bad and constant columns: [customerID]

██| (done) 100%


The next step is to look at the AutoML leaderboard and fetch the best model. When a leaderboard_frame is passed to the H2OAutoML.train() function, the models are evaluated and ranked using their performance on that particular dataset.<br>
like for Classification Problems metrices will likely be
<li>AUC / AUCPR</li>
<li>LogLoss</li>
<li>Confusion Matrix</li>
<li>F1 scores</li>
<li>Classification Thresholds</li>
where for Regression Problems metrics will differ like MSE, RMSE, R², AIC

In [12]:
best_model = churn_aml.leader
best_model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/3
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [13]:
churn_aml.leaderboard.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20250511_110305,0.829692,0.425462,0.639243,0.259651,0.371614,0.138097
GBM_1_AutoML_1_20250511_110305,0.827988,0.426824,0.639986,0.255745,0.370951,0.137605
GLM_1_AutoML_1_20250511_110305,0.824801,0.432584,0.627179,0.258187,0.375182,0.140761
XGBoost_1_AutoML_1_20250511_110305,0.81882,0.46783,0.611221,0.267757,0.386143,0.149106
XGBoost_2_AutoML_1_20250511_110305,0.806243,0.584584,0.594859,0.272821,0.443612,0.196791


In [14]:
pred=best_model.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [16]:
churn_perf=best_model.model_performance(test)
churn_perf

ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.13809719464863654
RMSE: 0.37161430899339243
LogLoss: 0.42546217210163123
AUC: 0.8296920404464985
AUCPR: 0.6392434866028408
Gini: 0.6593840808929969
Null degrees of freedom: 1399
Residual degrees of freedom: 1396
Null deviance: 1592.9207511071543
Residual deviance: 1191.2940818845675
AIC: 1199.2940818845675

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4345345054598353
       No    Yes    Error    Rate
-----  ----  -----  -------  --------------
No     888   154    0.1478   (154.0/1042.0)
Yes    133   225    0.3715   (133.0/358.0)
Total  1021  379    0.205    (287.0/1400.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.434535     0.610583  157
max f2                       0.188275     0.726516  274
max f0point5                 0.530351     0.620592  116
max accuracy                 0.530351     0.806429  116
max precision                0.801746     0.9375    9
max recall                   0.0107237    1         395
max specificity              0.845367     0.99904   0
max absolute_mcc             0.434535     0.471972  157
max min_per_class_accuracy   0.30239      0.737044  217
max mean_per_class_accuracy  0.260743     0.745411  238
max tns                      0.845367     1041      0
max fns                      0.845367     357       0
max fps                      0.00712945   1042      399
max tps                      0.0107237    358       395
max tnr                      0.845367     0.99904   0
max fnr                      0.845367     0.997207  0
max fpr                      0.00712945   1         399
max tpr                      0.0107237    1         395

Gains/Lift Table: Avg response rate: 25.57 %, avg score: 27.23 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.01                        0.805297           3.63128    3.63128            0.928571         0.823223   0.928571                    0.823223            0.0363128       0.0363128                  263.128   263.128            0.0353532
2        0.02                        0.775332           3.35196    3.49162            0.857143         0.788665   0.892857                    0.805944            0.0335196       0.0698324                  235.196   249.162            0.0669533
3        0.03                        0.760946           3.35196    3.44507            0.857143         0.768714   0.880952                    0.793534            0.0335196       0.103352                   235.196   244.507            0.0985535
4        0.04                        0.744077           3.35196    3.42179            0.857143         0.752973   0.875                       0.783394            0.0335196       0.136872                   235.196   242.179            0.130154
5        0.05                        0.731549           3.07263    3.35196            0.785714         0.7393     0.857143                    0.774575            0.0307263       0.167598                   207.263   235.196            0.158001
6        0.1                         0.652225           2.56983    2.96089            0.657143         0.689488   0.757143                    0.732031            0.128492        0.296089                   156.983   196.089            0.26346
7        0.15                        0.577139           2.12291    2.68156            0.542857         0.617879   0.685714 

max_runtime_secs set to be 60s

In [17]:
churn_aml_02=H2OAutoML(seed=42,max_runtime_secs=60,project_name="telco_churn_aml_02")
churn_aml_train=churn_aml_02.train(y=y,training_frame=train,leaderboard_frame=test)

AutoML progress: |
11:13:25.588: _train param, Dropping bad and constant columns: [customerID]

██████████████████
11:13:43.94: _train param, Dropping bad and constant columns: [customerID]

████
11:13:47.566: _train param, Dropping bad and constant columns: [customerID]

███████████
11:13:57.82: _train param, Dropping unused columns: [customerID]
11:13:57.999: _train param, Dropping bad and constant columns: [customerID]

██████
11:14:02.897: _train param, Dropping bad and constant columns: [customerID]

███
11:14:07.4: _train param, Dropping bad and constant columns: [customerID]

█████
11:14:11.43: _train param, Dropping bad and constant columns: [customerID]

█████
11:14:15.729: _train param, Dropping bad and constant columns: [customerID]

█████
11:14:19.842: _train param, Dropping unused columns: [customerID]
11:14:20.670: _train param, Dropping unused columns: [customerID]

██
11:14:21.963: _train param, Dropping bad and constant columns: [customerID]

█
11:14:23.217: _train par

In [18]:
churn_aml_02.leaderboard.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_2_AutoML_2_20250511_111325,0.83002,0.424796,0.643152,0.26432,0.37116,0.137759
StackedEnsemble_AllModels_1_AutoML_2_20250511_111325,0.829389,0.425366,0.642353,0.266591,0.371436,0.137965
StackedEnsemble_BestOfFamily_1_AutoML_2_20250511_111325,0.82888,0.425847,0.64026,0.265366,0.371627,0.138107
GBM_1_AutoML_2_20250511_111325,0.82683,0.427191,0.635522,0.256139,0.371732,0.138185
GLM_1_AutoML_2_20250511_111325,0.824855,0.432274,0.627487,0.258667,0.375136,0.140727
GBM_2_AutoML_2_20250511_111325,0.820933,0.434485,0.623346,0.253997,0.374441,0.140206
XGBoost_2_AutoML_2_20250511_111325,0.819076,0.465607,0.625421,0.259147,0.384883,0.148135
GBM_5_AutoML_2_20250511_111325,0.817814,0.541288,0.620853,0.253337,0.423566,0.179408
XGBoost_1_AutoML_2_20250511_111325,0.817326,0.440753,0.618153,0.262205,0.378728,0.143435
GBM_3_AutoML_2_20250511_111325,0.813908,0.442973,0.627171,0.260329,0.377509,0.142513


 Telco Customer Churn Prediction study, the reported AUC for Logistic Regression was 0.852, which is slightly higher than the best AUC achieved in y AutoML runs. This indicates that with more extensive training or fine-tuning, AutoML models could exceed the performance of traditional models keep in mind that time taken by AutoML is significantly lesser to train several models.
